In [7]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-144835
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-144835


In [8]:

from azureml.core.compute import ComputeTarget, AmlCompute
compute_cluster_name = "lr-cluster"
provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, compute_cluster_name, provisioning_config)
compute_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
        '--C': uniform(0.1, 0.9), '--max_iter': choice(50, 100, 150, 250, 300)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")
    shutil.copy("train.py", "./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = ".", entry_script = "train.py", compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     policy=policy,
                                     estimator=est,
                                     max_concurrent_runs=5,
                                     max_total_runs = 20,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE)

In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
hyperdrive_run = exp.submit(hyperdrive_config, showOutput=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [20]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n --C:',parameter_values[1])
print('\n --max_iter',parameter_values[3])
### YOUR CODE HERE ###

Best Run Id:  HD_b0e3d65a-7daa-4eea-80a6-73e9ca2df93f_1

 Accuracy: 0.9107044325915027

 --C: 0.17314110766498897

 --max_iter 100


In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_to_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path_to_data)

In [22]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=11)
training_data = x_train.join(y_train)

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [28]:
# Submit your automl run
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "automl_classification")
run = experiment.submit(automl_config, show_output = True)
RunDetails(run).show()

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
pandas<=0.25.3/pandas 1.2.4
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\npandas<=0.25.3/pandas 1.2.4",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
                "code": "IncompatibleOrMissingDependency"
            }
  

In [21]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()
print(best_run)

print(fitted_model)

Run(Experiment: automl_classification,
Id: AutoML_9aff4796-f858-4650-b19d-d9ce3b6f07cb_43,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=2.0833333333333335,
                                                                                               scale_pos_weight=1,
                             

In [23]:
from sklearn.metrics import mean_squared_error
from math import sqrt
y_predict = fitted_model.predict(x_test)
rmse = sqrt(mean_squared_error(y_test, y_predict))
print(rmse)

0.29039558110591396
